In [7]:
!pip install trl
!pip install -U bitsandbytes>=0.46.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash_attn


In [2]:
import os
import json
import re
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
import torch
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration

# ─────────────────────────────
# CONFIG
# ─────────────────────────────
IMAGE_DIR = r"/content/drive/MyDrive/data/images"
SFT_OUT = "/content/drive/MyDrive/data/sft/train.jsonl"
DPO_OUT = "/content/drive/MyDrive/data/dpo/train.jsonl"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VALID_EXT = (".jpg", ".jpeg", ".png", ".webp")

# ─────────────────────────────
# CREATE DIRS
# ─────────────────────────────
os.makedirs("data/sft", exist_ok=True)
os.makedirs("data/dpo", exist_ok=True)

# ─────────────────────────────
# LOAD BLIP
# ─────────────────────────────
print("Loading BLIP model...")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(DEVICE)

# ─────────────────────────────
# HELPERS
# ─────────────────────────────
def clean_caption(text: str) -> str:
    text = re.sub(r"\b(\w+)( \1\b)+", r"\1", text)  # remove repetition
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def generate_caption(img_path: Path) -> str:
    image = Image.open(img_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(DEVICE)
    output = model.generate(**inputs, max_new_tokens=30)
    caption = processor.decode(output[0], skip_special_tokens=True)
    return clean_caption(caption)

def strong_description(caption: str) -> str:
    return (
        f"{caption.capitalize()}. "
        "Clean visual appearance with good composition and clarity. "
        "Well-suited for professional listings and marketing use."
    )

def weak_description(caption: str) -> str:
    return (
        f"{caption.capitalize()}. "
        "Basic description with limited detail and no promotional context."
    )

# ─────────────────────────────
# MAIN LOOP
# ─────────────────────────────
images = [
    p for p in Path(IMAGE_DIR).iterdir()
    if p.suffix.lower() in VALID_EXT
]

print(f"Found {len(images)} images")

with open(SFT_OUT, "w", encoding="utf-8") as sft_f, \
     open(DPO_OUT, "w", encoding="utf-8") as dpo_f:

    for img in tqdm(images, desc="Building dataset"):
        caption = generate_caption(img)
        rel_path = f"images/{img.name}"

        chosen = strong_description(caption)
        rejected = weak_description(caption)

        # ───── SFT ─────
        sft_sample = {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Create a short, SEO-friendly product description based on the image."},
                        {"type": "image", "image": rel_path}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": chosen}
                    ]
                }
            ]
        }
        sft_f.write(json.dumps(sft_sample) + "\n")

        # ───── DPO ─────
        dpo_sample = {
            "prompt": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Create a short, SEO-friendly product description based on the image."},
                        {"type": "image", "image": rel_path}
                    ]
                }
            ],
            "chosen": chosen,
            "rejected": rejected
        }
        dpo_f.write(json.dumps(dpo_sample) + "\n")

print("✅ Clean SFT and DPO datasets created")
print(f"→ {SFT_OUT}")
print(f"→ {DPO_OUT}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading BLIP model...


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

The image processor of type `BlipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie text_decoder.cls.predictions.bias to text_decoder.cls.predictions.decoder.bias, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie text_decoder.bert.embeddings.word_embeddings.weight to text_decoder.cls.predictions.decoder.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
BlipForConditionalGeneration LOAD REPORT from: Salesforce/blip-image-captioning-base
Key                                       | Status     |  | 
------------------------------------------+------------+--+-
text_decoder.bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identic

Found 50 images


Building dataset: 100%|██████████| 50/50 [00:36<00:00,  1.35it/s]

✅ Clean SFT and DPO datasets created
→ /content/drive/MyDrive/data/sft/train.jsonl
→ /content/drive/MyDrive/data/dpo/train.jsonl


In [ ]:
import torch
from google.colab import drive
drive.mount('/content/drive')
from datasets import load_dataset
from transformers import (
    AutoProcessor,
    TrainingArguments,
    Trainer,
    Qwen2VLForConditionalGeneration,
    BitsAndBytesConfig # Import BitsAndBytesConfig
)
from trl import DPOTrainer, DPOConfig
from PIL import Image
import os
from pathlib import Path # Added import for Path

# Define IMAGE_DIR here as it's used in data collators
IMAGE_DIR = r"/content/drive/MyDrive/data/images"

MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"

# ────────────────────────────
# LOAD PROCESSOR + MODEL (CORRECT)
# ────────────────────────────
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quantization_config, # Apply quantization
    trust_remote_code=True
)

# ────────────────────────────
# SFT DATA COLLATOR
# ────────────────────────────
def vl_data_collator(batch):
    texts, images = [], []

    for sample in batch:
        messages = sample["messages"]

        user_content = messages[0]["content"]
        instruction = next(x["text"] for x in user_content if x["type"] == "text")
        image_path = next(x["image"] for x in user_content if x["type"] == "image")
        answer = messages[1]["content"][0]["text"]

        chat = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": instruction}
                ]
            },
            {
                "role": "assistant",
                "content": answer
            }
        ]

        text = processor.apply_chat_template(
            chat,
            tokenize=False,
            add_generation_prompt=False
        )

        texts.append(text)
        # Corrected image path: Use IMAGE_DIR to build the full path
        images.append(Image.open(os.path.join(IMAGE_DIR, Path(image_path).name)).convert("RGB"))

    inputs = processor(
        text=texts,
        images=images,
        padding=True,
        return_tensors="pt"
    )

    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

# ────────────────────────────
# SFT TRAINING
# ────────────────────────────
print("\n🔥 Starting SFT training...\n")

sft_dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/data/sft/train.jsonl",
    split="train"
)

sft_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/data/outputs/",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    logging_steps=5,
    bf16=torch.cuda.is_available(),
    gradient_checkpointing=True, # Added for memory optimization
    remove_unused_columns=False,
    report_to="none"
)

sft_trainer = Trainer(
    model=model,
    args=sft_args,
    train_dataset=sft_dataset,
    data_collator=vl_data_collator
)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
sft_trainer.train()
sft_trainer.save_model("/content/drive/MyDrive/data/outputs/")

print("\n✅ SFT completed\n")


# ────────────────────────────
# LOAD SFT MODEL FOR DPO
# ────────────────────────────
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "/content/drive/MyDrive/data/outputs/", # Corrected path to the saved SFT model
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

# ────────────────────────────
# DPO DATA COLLATOR
# ────────────────────────────
def dpo_vl_collator(batch):
    prompts, chosen, rejected, images = [], [], [], []

    for sample in batch:
        user = sample["prompt"][0]["content"]
        instruction = next(x["text"] for x in user if x["type"] == "text")
        image_path = next(x["image"] for x in user if x["type"] == "image")

        chat = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": instruction}
                ]
            }
        ]

        prompt = processor.apply_chat_template(
            chat,
            tokenize=False,
            add_generation_prompt=True
        )

        prompts.append(prompt)
        chosen.append(sample["chosen"])
        rejected.append(sample["rejected"])
        # Corrected image path: Use IMAGE_DIR to build the full path
        images.append(Image.open(os.path.join(IMAGE_DIR, Path(image_path).name)).convert("RGB"))

    return {
        "prompt": prompts,
        "chosen": chosen,
        "rejected": rejected,
        "images": images,
    }

# ────────────────────────────
# DPO TRAINING
# ────────────────────────────
print("\n🔥 Starting DPO training...\n")

dpo_dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/data/dpo/train.jsonl",
    split="train"
)

dpo_args = DPOConfig(
    output_dir="/content/drive/MyDrive/data/outputs/",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    beta=0.1,
    logging_steps=5,
    bf16=torch.cuda.is_available(),
    gradient_checkpointing=True, # Added for memory optimization
    remove_unused_columns=False,
)

dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_args,
    train_dataset=dpo_dataset,
    data_collator=dpo_vl_collator
)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
dpo_trainer.train()
dpo_trainer.save_model("/content/drive/MyDrive/data/outputs/")

print("\n🎉 SFT → DPO pipeline finished successfully")